# Flower Image Classification Application 
The aim of this project is to develop a fully trained, neural network able to accurately classify a range of 21 different flower species. This lab is being done as part of the Udacity PyTorch Scholarship Challlenge This will be done using PyTorch. 

In [37]:
# Imports
import torch
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


## Loading & Pre-Processing Data

### Image Augmentation
Below we define a set of transformations, including rotation, cropping and horizontal flip. We do this to prevent our model making predictions based on the location, size, and orientation of an object within the image. These are only applied to the training data, as they'll be used to train the model. We also crop the images to our desired size and convert them into tensors, allowing them to be transffered to GPU for accelerated training. 

For this classifier we're using a pre-trained model that was previously trained on an ImageNet dataset. These images were normalized and therefore we must also add a normalization transformation prior to training. 

In [38]:
# Define your transforms for the training and validation sets
train_data_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


### Creating the Dataloader
Below we load our images from our folder using *tourchvision's dataset* function and apply our transformations. We also specify a batch size that'll be used in our neural networks. 

In [39]:
data_dir = '/floyd/input/flower_dataset/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

# Batch Size
batch_size = 20;

# Load the datasets with ImageFolder
train_image_dataset = datasets.ImageFolder(train_dir, transform=train_data_transforms)
valid_image_dataset = datasets.ImageFolder(valid_dir, transform=test_data_transforms)

# Using the image datasets and the trainforms, define the dataloaders
train_dataloader = torch.utils.data.DataLoader(train_image_dataset, batch_size=batch_size, shuffle=True) 
valid_dataloader = torch.utils.data.DataLoader(valid_image_dataset, batch_size=batch_size, shuffle=True) 

### Label Mapping
Below we import the supplied cat_to_name json document that allows us to create a mapping of category id to label.   

In [40]:
import json
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

## Importing & Defining the Model
This model uses transfer learning where the pre-trainined model is the VGG16 pre-training network. This model is downloaded and then feature parameters are frozen.

In [41]:
# Import Pre-trained Model
model = models.vgg16(pretrained=True)

In [42]:
# Freeze Parameters
for param in model.parameters():
    param.requires_grad = False

In [43]:
# Define the new classsifier
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088,12544)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(.2)),
    ('fc2', nn.Linear(12544,6272)),
    ('relu2', nn.ReLU()),
    ('dropout2', nn.Dropout(.2)),
    ('fc3', nn.Linear(6272,3136)),
    ('relu3', nn.ReLU()),
    ('dropout3', nn.Dropout(.2)),
    ('fc4', nn.Linear(3136,102)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

KeyboardInterrupt: 

## Training the Model

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# number of epochs to train the model
n_epochs = 30

# tracking arrays for validation and training loss
train_losses, valid_losses = [], []

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    train_losses.append(running_loss/len(train_dataloader))
    valid_losses.append(valid_loss/len(test_dataloader))
    
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        model.class_to_idx = train_image_dataset.class_to_idx
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss